In [4]:
from alpha_vantage.timeseries import TimeSeries

def fetch_stock_data_alpha_vantage(ticker, api_key="1H4BLOX4T4ZVGAED"):
    ts = TimeSeries(key=api_key, output_format='pandas')
    try:
        data, meta_data = ts.get_daily(symbol=ticker, outputsize='compact')
        data.reset_index(inplace=True)
        return data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

stock_data = fetch_stock_data_alpha_vantage("AAPL", api_key="1H4BLOX4T4ZVGAED")
print(stock_data.head())

        date  1. open   2. high    3. low  4. close   5. volume
0 2024-11-27  234.465  235.6900  233.8101    234.93  33498439.0
1 2024-11-26  233.330  235.5700  233.3300    235.06  45986189.0
2 2024-11-25  231.460  233.2450  229.7400    232.87  90152832.0
3 2024-11-22  228.060  230.7199  228.0600    229.87  38168252.0
4 2024-11-21  228.880  230.1550  225.7103    228.52  42108327.0


In [6]:
def calculate_rsi(data, period=14):
    """
    Calculate the Relative Strength Index (RSI) for a given dataset.
    :param data: DataFrame with a 'close' column
    :param period: Look-back period for RSI (default: 14)
    :return: DataFrame with an additional 'RSI' column
    """
    # Calculate daily price changes
    data["change"] = data["4. close"].diff()
    
    # Separate gains and losses
    data["gain"] = data["change"].apply(lambda x: x if x > 0 else 0)
    data["loss"] = data["change"].apply(lambda x: -x if x < 0 else 0)
    
    # Calculate average gain and loss
    data["avg_gain"] = data["gain"].rolling(window=period, min_periods=1).mean()
    data["avg_loss"] = data["loss"].rolling(window=period, min_periods=1).mean()
    
    # Calculate Relative Strength (RS)
    data["rs"] = data["avg_gain"] / data["avg_loss"]
    
    # Calculate RSI
    data["RSI"] = 100 - (100 / (1 + data["rs"]))
    
    return data

# Calculate RSI for the stock data
stock_data_with_rsi = calculate_rsi(stock_data)
stock_data_with_rsi.tail()

,date,1. open,2. high,3. low,4. close,5. volume,change,gain,loss,avg_gain,avg_loss,rs,RSI
95,2024-07-16,235.00,236.27,232.33,234.82,43234278.0,5.94,5.94,0.00,2.344286,0.519286,4.514443,81.865802
96,2024-07-15,236.48,237.23,233.09,234.40,62631252.0,-0.42,0.00,0.42,1.587857,0.549286,2.890767,74.298128
97,2024-07-12,228.92,232.64,228.68,230.54,53046527.0,-3.86,0.00,3.86,1.587857,0.717857,2.211940,68.866171
98,2024-07-11,231.39,232.39,225.77,227.57,64710617.0,-2.97,0.00,2.97,1.322143,0.930000,1.421659,58.705994
99,2024-07-10,229.30,233.08,229.25,232.98,62627687.0,5.41,5.41,0.00,1.708571,0.695714,2.455852,71.063577


In [8]:
import requests
import pandas as pd

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=ABCB&time_from=20241001T0130&apikey=1H4BLOX4T4ZVGAED'
r = requests.get(url)
data = r.json()
df = pd.DataFrame(data)
df


,items,sentiment_score_definition,relevance_score_definition,feed
0,8,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",{'title': 'Implied Volatility Surging for Amer...
1,8,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",{'title': 'Ameris Bancorp ( ABCB ) Q3 Earnin...
2,8,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",{'title': 'Thursday's big stock stories: What'...
3,8,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...","{'title': 'South Atlantic Bancshares, Inc. ( ..."
4,8,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",{'title': 'Ameris Bancorp ( ABCB ) Q3 Earnin...
5,8,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",{'title': 'Chemung Financial ( CHMG ) Surpas...
6,8,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",{'title': 'Ameris Bancorp ( ABCB ) Earnings ...
7,8,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",{'title': 'Why Ameris Bancorp ( ABCB ) is Po...


In [10]:
data

{'items': '8',
 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish',
 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.',
 'feed': [{'title': 'Implied Volatility Surging for Ameris Bancorp  ( ABCB )  Stock Options',
   'url': 'https://www.zacks.com/stock/news/2359553/implied-volatility-surging-for-ameris-bancorp-abcb-stock-options',
   'time_published': '20241029T134100',
   'authors': ['Zacks Equity Research'],
   'summary': 'Investors need to pay close attention to Ameris Bancorp (ABCB) stock based on the movements in the options market lately.',
   'banner_image': 'https://staticx-tuner.zacks.com/images/articles/main/b6/2622.jpg',
   'source': 'Zacks Commentary',
   'category_within_source': 'n/a',
   'source_domain': 'www.zacks.com',
   'topics': [{'topic': 'Earnings', 'relevance_score': '0.158519'},
    {'topic': 'Financ

In [12]:

abcb_sentiment_scores = []

# Iterate through all of the articles in the feed.
for item in data.get('feed', []):
    # Check if there is a 'ticker_sentiment' field in the current feed item.
    if 'ticker_sentiment' in item:
        for ticker_data in item['ticker_sentiment']:
            # Check if the ticker is ABCB.
            if ticker_data.get('ticker') == 'ABCB':
                # Get the sentiment score and label.
                score = float(ticker_data.get('ticker_sentiment_score', 0))
                label = ticker_data.get('ticker_sentiment_label', 'Unknown')
                abcb_sentiment_scores.append({'score': score, 'label': label})

# Print
    print("Sentiment scores for ABCB:")
    for sentiment in abcb_sentiment_scores:
        print(f"Score: {sentiment['score']}, Label: {sentiment['label']}")
else:
    print("No sentiment data found for ABCB.")

Sentiment scores for ABCB:
Score: 0.078406, Label: Neutral
Score: 0.14658, Label: Neutral
Score: -0.139079, Label: Neutral
Score: 0.0, Label: Neutral
Score: 0.264438, Label: Somewhat-Bullish
Score: 0.0, Label: Neutral
Score: 0.218595, Label: Somewhat-Bullish
Score: 0.438957, Label: Bullish
